using the in-built model in pytorch

on MNIST dataset

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda')


In [2]:
# input_size = 784    #28 x 28
hidden_size = 128
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

# using RNN to classify images means to input a image row by row, treat the rows as a seq.
input_size = 28
sequence_length = 28
num_layers = 2

In [3]:
# load datasets

train_dataset = torchvision.datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=False)

test_dataset = torchvision.datasets.MNIST(root='./data/', train=False, transform=transforms.ToTensor())

In [4]:
train_dataset.data.shape

torch.Size([60000, 28, 28])

In [5]:
# create data loader

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
# rnn model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True) # treat the batch_size as the first dim
        # x.shape -> (batch_size, seq, input_size)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # rnn need 2 inputs: x and init_hidden
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.rnn(x, h0)
        # out: batch_size, seq_length, hidden_size
        # decode the hidden_state only for the last step
        # out: N, 28, 128
        out = out[:, -1, :]
        # out: N, 128
        out = self.fc(out)
        return out


In [7]:
# GRU model
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True) # treat the batch_size as the first dim
        # x.shape -> (batch_size, seq, input_size)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # rnn need 2 inputs: x and init_hidden
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.gru(x, h0)
        # out: batch_size, seq_length, hidden_size
        # decode the hidden_state only for the last step
        # out: N, 28, 128
        out = out[:, -1, :]
        # out: N, 128
        out = self.fc(out)
        return out


In [8]:
# GRU model
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRU, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True) # treat the batch_size as the first dim
        # x.shape -> (batch_size, seq, input_size)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # rnn need 2 inputs: x and init_hidden
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.gru(x, h0)
        # out: batch_size, seq_length, hidden_size
        # decode the hidden_state only for the last step
        # out: N, 28, 128
        out = out[:, -1, :]
        # out: N, 128
        out = self.fc(out)
        return out


In [9]:
# LSTM model: needs additional cell state
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) # treat the batch_size as the first dim
        # x.shape -> (batch_size, seq, input_size)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # rnn need 2 inputs: x and init_hidden
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        # out: batch_size, seq_length, hidden_size
        # decode the hidden_state only for the last step
        # out: N, 28, 128
        out = out[:, -1, :]
        # out: N, 128
        out = self.fc(out)
        return out


In [10]:
rnn = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)

In [12]:
# training loop

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader): 
        # batch_size, 28, 28, doesn't need a reshape
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        #forward pass
        output = rnn(images)
        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % batch_size == 0:
            print(f'Epoch {epoch+1} / {num_epochs}, Step [{i+1} / {n_total_steps}], LOss: {loss.item():.4f}')


Epoch 1 / 2, Step [100 / 600], LOss: 0.7060
Epoch 1 / 2, Step [200 / 600], LOss: 0.5439
Epoch 1 / 2, Step [300 / 600], LOss: 0.5094
Epoch 1 / 2, Step [400 / 600], LOss: 0.1605
Epoch 1 / 2, Step [500 / 600], LOss: 0.1949
Epoch 1 / 2, Step [600 / 600], LOss: 0.0979
Epoch 2 / 2, Step [100 / 600], LOss: 0.1006
Epoch 2 / 2, Step [200 / 600], LOss: 0.1996
Epoch 2 / 2, Step [300 / 600], LOss: 0.1007
Epoch 2 / 2, Step [400 / 600], LOss: 0.0347
Epoch 2 / 2, Step [500 / 600], LOss: 0.2010
Epoch 2 / 2, Step [600 / 600], LOss: 0.0421


In [15]:
# eval loop
# prediction
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = rnn(images)

        # max returns (value, index)
        _, preds = torch.max(outputs, 1)
        n_samples += labels.size(0) # labels: [batch_size, 1]
        n_correct += (preds == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = preds[i]
            if label == pred :
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {train_dataset.classes[i]}: {acc} %')

Accuracy: 97.44 %
Accuracy of 0 - zero: 97.24489795918367 %
Accuracy of 1 - one: 99.64757709251101 %
Accuracy of 2 - two: 97.18992248062015 %
Accuracy of 3 - three: 96.93069306930693 %
Accuracy of 4 - four: 97.35234215885947 %
Accuracy of 5 - five: 97.19730941704036 %
Accuracy of 6 - six: 98.43423799582463 %
Accuracy of 7 - seven: 96.40077821011673 %
Accuracy of 8 - eight: 98.56262833675565 %
Accuracy of 9 - nine: 95.24281466798811 %
